In [1]:
import requests
from bs4 import BeautifulSoup
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import pandas as pd
import json

from functions import (get_sector, changes_from_press, get_df, get_content_header,
                       normalise_to_index, save_data)

past_last_date = False

C:\Users\Aidan\miniconda3\envs\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = []
offset = 0

ticker = 'snps'
sector = get_sector(ticker)

stock_hist = get_df(ticker)
index_ticker = '^GSPC'
# select by value 

stock_hist = get_df(ticker)
index_hist = get_df(index_ticker)


# create break
while True:
    
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36"}
    
    s = requests.Session()
    html = requests.get(f'https://news.synopsys.com/index.php?s=20295&l=400&o={offset}', headers=headers)
    

    soup = BeautifulSoup(html.content) 

    articles = soup.find_all('div',attrs={'wd_item_wrapper'})
 
    if not articles:
        break

    for article in articles:
        date = article.find('div', attrs={'wd_date'}).text.lstrip().rstrip()
        title = article.find('a').text.lstrip().rstrip() 
        url = article.find('a').get('href')
    
        
        #date = date.replace(',', '')
    
        date = datetime.strptime(date, '%b %d, %Y').date()
        content = get_content_header(url, 'meta', {'property':'og:description'})
        if date < datetime(2014, 1, 1).date():
            past_last_date = True
            break
        
        pct_change = changes_from_press(stock_hist, date, 1)
        
        if pct_change != None:
            index_price, norm_price = normalise_to_index(pct_change, date, index_hist)
        else:
            index_price, norm_price = (None, None)

        data.append([str(date), title, url, content, pct_change, index_price, norm_price])
    if date < datetime(2014, 1, 1).date(): break    
    offset = offset + 400
    
save_data(data,ticker, sector)